In [75]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
data = pd.read_sas(r"\\10.1.1.58\datos\CHILE\IMPORTER_POST_VENTA_CONTRATO_MANTENIMIENTO\detalle_chassis.sas7bdat")
data.head()

,ACUM_FALLAS_CORRECTIVAS,ACUM_ING,ACUM_MGN,NUM_CONTR,ORDEN,CLIENTE,INICIO,TERMINO,DURACION_CM,ANTIGUEDAD_CM,...,COLOR_POST_VENTA,COLOR_VENTA,ING,COS,MGN,RENT,RENT_ACUM,PERMANENCIA,DISPONIBILIDAD_MES,DISPONIBILIDAD_ACUM
0,0.0,65789.0,65789.0,b'0040001141',b'CM-40001141',b'0001128597',2012-12-05,2019-03-31,75.0,72.0,...,b'VERDE',b'AMARILLO',65789.0,0.0,65789.0,1.0,1.0,0.0,1.0,1.0
1,0.0,414258.0,414258.0,b'0040001141',b'CM-40001141',b'0001128597',2012-12-05,2019-03-31,75.0,72.0,...,b'VERDE',b'AMARILLO',348469.0,0.0,348469.0,1.0,1.0,0.0,1.0,1.0
2,1.0,414258.0,414258.0,b'0040001141',b'CM-40001141',b'0001128597',2012-12-05,2019-03-31,75.0,72.0,...,b'VERDE',b'AMARILLO',0.0,0.0,0.0,NaN,1.0,0.0,1.0,1.0
3,1.0,598168.0,598168.0,b'0040001141',b'CM-40001141',b'0001128597',2012-12-05,2019-03-31,75.0,72.0,...,b'VERDE',b'AMARILLO',183910.0,0.0,183910.0,1.0,1.0,0.0,1.0,1.0
4,1.0,618791.0,618791.0,b'0040001141',b'CM-40001141',b'0001128597',2012-12-05,2019-03-31,75.0,72.0,...,b'VERDE',b'AMARILLO',20623.0,0.0,20623.0,1.0,1.0,0.0,1.0,1.0


In [81]:
data=data.dropna()

In [155]:
x=data.drop(["CLIENTE","INICIO","TERMINO",'DURACION_CM',"CHASSIS","NUM_CONTR","ORDEN","ANIO","MES","ACUM_MGN","ACUM_MGN","RENT",'RENT_ACUM'], axis=1)
x=x.drop(['MGN','PERMANENCIA','DISPONIBILIDAD_MES','ING','COS','ACUM_ING','FALLAS_CORRECTIVAS','ANTIGUEDAD_CM'], axis=1)
y=data['RENT_ACUM']

In [156]:
def variables_dummy(dataset,variable):
    cat_dummies = pd.get_dummies(dataset[variable],prefix=variable)
    #.iloc[:,1:]
    data_new = dataset.join(cat_dummies)
    dataset = data_new.drop([variable],axis=1)
    return dataset

In [158]:
x.columns

Index(['ACUM_FALLAS_CORRECTIVAS', 'COBERTURA', 'SUB_CATEGORIA', 'MARCA',
       'TRABAJO', 'OPERACION', 'SERVICIO', 'ANTIGUEDAD', 'FLOTA_KF',
       'FLOTA_NOKF', 'COLOR_POST_VENTA', 'COLOR_VENTA', 'DISPONIBILIDAD_ACUM'],
      dtype='object')

In [159]:
tabla=variables_dummy(x,"COBERTURA")
tabla=variables_dummy(tabla,"SUB_CATEGORIA")
tabla=variables_dummy(tabla,"MARCA")
tabla=variables_dummy(tabla,"TRABAJO")
tabla=variables_dummy(tabla,"OPERACION")
tabla=variables_dummy(tabla,"SERVICIO")
tabla=variables_dummy(tabla,"COLOR_POST_VENTA")
tabla=variables_dummy(tabla,'COLOR_VENTA')

In [160]:
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  AdaBoostRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics 

import warnings
import numpy as np

In [161]:
x_train,x_test,y_train,y_test=train_test_split(tabla,y)

In [165]:
modelos = [["LM","RIDGE","BRIDGE","TREE","RandomForest","GBOOST","ADABOOST","MLPREGRESOR"],
           [LinearRegression(fit_intercept=True, normalize=True),
            Ridge(),
            BayesianRidge(),
            DecisionTreeRegressor(),
            RandomForestRegressor(),
            GradientBoostingRegressor(),
            AdaBoostRegressor(),
            MLPRegressor(hidden_layer_sizes=(60,),  activation='tanh', solver='lbfgs', alpha=0.00001)]
          ]
warnings.filterwarnings("ignore")

In [166]:
resultados=[]
predicion=[]
error=[]
precision=[]
a=0
for i in modelos[1]:
    ajuste=i.fit(x_train,y_train)
    resultados.append(ajuste)
    predicion.append(ajuste.predict(x_test))
    er = np.sqrt(metrics.mean_squared_error(y_test,ajuste.predict(x_test)))
    error.append(er)
    precision.append(ajuste.score(x_test , y_test))
    print("modelo:",modelos[0][a])
    print("ajuste de:", ajuste.score(x_test , y_test))
    a=a+1

modelo: LM
ajuste de: 0.08343021704612785
modelo: RIDGE
ajuste de: 0.08345538232785521
modelo: BRIDGE
ajuste de: 0.08347704886532525
modelo: TREE
ajuste de: 0.08405514937985248
modelo: RandomForest
ajuste de: 0.3438323199725068
modelo: GBOOST
ajuste de: 0.27877285595109735
modelo: ADABOOST
ajuste de: -0.35101205516534356
modelo: MLPREGRESOR
ajuste de: 0.10326796463190213


In [162]:
resumen=pd.DataFrame(list(zip(np.array(modelos[0]).T.tolist(),error,precision)))
resumen.columns=["Modelo","Error","Precision"]
resumen

,Modelo,Error,Precision
0,LM,2.655799,0.066755
1,RIDGE,2.655804,0.066752
2,BRIDGE,2.655901,0.066683
3,TREE,2.655813,0.066745
4,RandomForest,2.414835,0.228422
5,GBOOST,2.333765,0.279358
6,ADABOOST,3.171839,-0.331152
7,MLPREGRESOR,2.628592,0.085778


In [167]:
#busco maxima precision
max_pre=resumen.max(axis=0)["Precision"]
#busco modelo
selec=resumen[resumen["Precision"]==max_pre]

In [168]:
a=selec["Modelo"]
for i,w in zip(modelos[0],modelos[1]):
    for j in selec["Modelo"]:
        if i == j:
            modelo_final = w

In [169]:
modelo_final

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [170]:
importancia = pd.DataFrame(list(zip(x_train.columns, modelo_final.feature_importances_)))
importancia.sort_values(1,ascending=False)

,0,1
1,ANTIGUEDAD,0.258863
4,DISPONIBILIDAD_ACUM,0.134994
3,FLOTA_NOKF,0.127018
0,ACUM_FALLAS_CORRECTIVAS,0.097658
2,FLOTA_KF,0.095706
18,MARCA_b'FUSO',0.062005
7,SUB_CATEGORIA_b'CAMIONES 4X2 - 6X2',0.052839
28,OPERACION_b'DISTRIBUCION DE REFRESCOS',0.021424
36,OPERACION_b'POLIBRAZO',0.017367
12,SUB_CATEGORIA_b'TAXIBUS',0.016169
